In [1]:
# Imports

import time

Diplomarbeit Kaffeevollautomat

Datenschema + Demodaten

Betriebsbereite Maschine

Maschine reinigt sich 0:02 - 0:57

0:22 - 0:27 Brühgruppe fährt nach oben
0:33 - 0:51 Wasser läuft durch die Maschine und kommt vorne wieder raus
0:51 - 0:56 Brühgruppe fährt wieder nach unten

Betriebsbereite Maschine

Kaffee rausdrücken 1:04 - 1:42

1:04 - Knopf wird gedrückt
1:04 - 1:10 Kaffee wird gemahlen
1:10 - 1:15 Brühgruppe fährt nach oben / Gemahlener Kaffee wird gepresst
1:18 - 1:20 Pulver wird kurz angefeuchtet und die Temperatur des Wassers wird reguliert
1:24 - 1:38 Kaffee wird gebrüht
1:38 - Kaffee fertig zur Entnahme
1:38 - 1:42 Brühgruppe fährt in Ausgangsposition

Demodaten Kaffee rausdrücken

In [2]:
# Temperaturverlauf

# Kaffee mahlen

class CoffeeGrind:
    def __init__(self, second: int, description: str):
        self.second = second
        self.description = description

    def __repr__(self):
        return f"Sekunde {self.second}: {self.description}"

grinding = [
    CoffeeGrind(second=i + 1, description="Kaffee wird gemahlen")
    for i in range(6)
]

# Brühgruppe hochfahren / pressen

class CoffeePress:
    def __init__(self, second: int, description: str):
        self.second = second
        self.description = description

    def __repr__(self):
        return f"Sekunde {self.second}: {self.description}"

press = [
    CoffeePress(second=i + 1, description="Kaffee wird gepresst")
    for i in range(5)
]

# Anfeuchten / Regulieren

class PowderMoister:
    def __init__(self, second: int, description: str):
        self.second = second
        self.description = description

    def __repr__(self):
        return f"Sekunde {self.second}: {self.description}"

moisting = [
    CoffeePress(second=i + 1, description="Kaffee wird angefeuchtet")
    for i in range(2)
]

# Brühen 

class CoffeeBrewing:
    def __init__(self, second: int, description: str):
        self.second = second
        self.description = description

    def __repr__(self):
        return f"Sekunde {self.second}: {self.description}"

brewing = [
    CoffeeBrewing(second=i + 1, description="Kaffee wird gebrüht")
    for i in range(14)
]

# Brühgruppe runterfahren

class BrewingUnitToStartPosition:
    def __init__(self, second: int, description: str):
        self.second = second
        self.description = description

    def __repr__(self):
        return f"Sekunde {self.second}: {self.description}"

toStartPosition = [
    BrewingUnitToStartPosition(second=i + 1, description="Brüheinheit fährt in Startposition")
    for i in range(4)
]

Durchlauf von einem Kaffee der runtergedrückt wird

In [6]:
startProcess = input("Kaffee runterlassen (1) ")

if startProcess := 1:
    for sec in grinding:
        print(sec)
        time.sleep(1)
    for sec in press:
        print(sec)
        time.sleep(1)
    time.sleep(3)
    for sec in moisting:
        print(sec)
        time.sleep(1)
    time.sleep(4)
    for sec in brewing:
        print(sec)
        time.sleep(1)
    print("Kaffee ist fertig")
    time.sleep(1)
    for sec in toStartPosition:
        print(sec)
        time.sleep(1)


Sekunde 1: Kaffee wird gemahlen
Sekunde 2: Kaffee wird gemahlen
Sekunde 3: Kaffee wird gemahlen
Sekunde 4: Kaffee wird gemahlen
Sekunde 5: Kaffee wird gemahlen
Sekunde 6: Kaffee wird gemahlen
Sekunde 1: Kaffee wird gepresst
Sekunde 2: Kaffee wird gepresst
Sekunde 3: Kaffee wird gepresst
Sekunde 4: Kaffee wird gepresst
Sekunde 5: Kaffee wird gepresst
Sekunde 1: Kaffee wird angefeuchtet
Sekunde 2: Kaffee wird angefeuchtet
Sekunde 1: Kaffee wird gebrüht
Sekunde 2: Kaffee wird gebrüht
Sekunde 3: Kaffee wird gebrüht
Sekunde 4: Kaffee wird gebrüht
Sekunde 5: Kaffee wird gebrüht
Sekunde 6: Kaffee wird gebrüht
Sekunde 7: Kaffee wird gebrüht
Sekunde 8: Kaffee wird gebrüht
Sekunde 9: Kaffee wird gebrüht
Sekunde 10: Kaffee wird gebrüht
Sekunde 11: Kaffee wird gebrüht
Sekunde 12: Kaffee wird gebrüht
Sekunde 13: Kaffee wird gebrüht
Sekunde 14: Kaffee wird gebrüht
Kaffee ist fertig
Sekunde 1: Brüheinheit fährt in Startposition
Sekunde 2: Brüheinheit fährt in Startposition
Sekunde 3: Brüheinheit fähr